In [2]:
from reward_predictor import Reward_Predictor
from pref_db import PrefBuffer, PrefDB, Segment

import numpy as np
import torch

In [3]:
db_val = PrefDB(maxlen=100)
for i in range(10):
    db_val.append(torch.rand((32, 1 ,84, 84), dtype=torch.float), torch.rand((32, 1 ,84, 84), dtype=torch.float), 1)
    
db_train = PrefDB(maxlen=100)
for i in range(10):
    db_train.append(torch.rand((32, 1 ,84, 84), dtype=torch.float), torch.rand((32, 1 ,84, 84), dtype=torch.float), 0)

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

from a2c_ppo_acktr.envs import make_vec_envs
envs = make_vec_envs('PongNoFrameskip-v4', 42, 8,
                     0.99, '/tmp/gym/', device, False)

reward_predictor = Reward_Predictor(
    envs.observation_space.shape
)
reward_predictor.to(device)
print('cudaa')
#reward_predictor(s1s.cuda(), s2s.cuda())

Logging to /tmp/openai-2020-11-21-16-49-47-344020
Creating dummy env object to get spaces
cudaa


In [ ]:
for i in range(100):
    reward_predictor.train(db_train, db_val, device)

In [6]:
o

In [6]:
#def train(self, pref_buffer):
def train(train_db, val_db, device):
    #train_db, val_db = pref_buffer.get_dbs()
    train_loader = torch.utils.data.DataLoader(
        train_db,
        batch_size=32,
        shuffle=False,
        num_workers=8
    )
    val_loader = torch.utils.data.DataLoader(
        val_db,
        batch_size=32,
        shuffle=False,
        num_workers=8
    )

    for batch in train_loader:
        s1s, s2s, prefs = batch

        reward_predictor.optimizer.zero_grad()

        preds = reward_predictor(s1s.to(device), s2s.to(device))

        loss = reward_predictor.criterion(preds, prefs.to(device))
        loss.backward()
        reward_predictor.optimizer.step()

    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            s1s, s2s, prefs = batch
            preds = reward_predictor(s1s.to(device), s2s.to(device))

            predicted = torch.max(preds.data, 1)[1]

            total += predicted.size(0)
            correct += (predicted == prefs.to(device)).sum().item()

    print(f'accuracy of reward on {total} trajectories: {(100 * correct / total)}')
    
train(db_train, db_val, device)

accuracy of reward on 10 trajectories: 60.0


In [17]:
train(db_train, db_val, device)

accuracy of reward on 10 trajectories: 50.0


In [101]:
prefs = torch.rand(4,2)
preds.data

tensor([[0.5016, 0.4984],
        [0.5023, 0.4977],
        [0.4980, 0.5020],
        [0.4993, 0.5007]], device='cuda:0')

In [95]:
_, predicted = torch.max(preds.data, 1)
predicted

tensor([0, 0, 1, 1], device='cuda:0')

In [85]:
_, predicted = torch.max(preds.data, 1)
predicted

tensor([0, 0, 1, 1], device='cuda:0')

In [104]:
torch.equal(torch.max(preds.data, 1)[1])

RuntimeError: Expected object of device type cuda but got device type cpu for argument #2 'other' in call to _th_equal

In [118]:
labels = torch.max(prefs.data, 1)[1]
predicted = torch.max(preds.data, 1)[1]

total += predicted.size(0)
correct += (predicted == labels.cuda()).sum().item()

3

In [70]:
torch.equal(torch.argmax(prefs, 1), torch.argmax(preds, 1))

from a2c_ppo_acktr.envs import make_vec_envs
import garner as g

g.login('kipgparker@gmail.com')
g.select_pool('Deep reinforcement learning from human prefrences')

In [12]:
pref_db_train = PrefDB(maxlen=20000)
pref_db_val = PrefDB(maxlen=5000)
pref_buffer = PrefBuffer(garner = g, db_train=pref_db_train,
                         db_val=pref_db_val, maxlen=30)

Creating dummy env object to get spaces


batchsize: 32
seg len: 32
shape: torch.Size([2048, 1, 84, 84])


tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]], device='cuda:0', grad_fn=<SoftmaxBackward>)

CNNBase(
  (main): Sequential(
    (0): Conv2d(1, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
    (6): Flatten()
    (7): Linear(in_features=1568, out_features=512, bias=True)
    (8): ReLU()
  )
  (critic_linear): Linear(in_features=512, out_features=1, bias=True)
)

In [26]:
envs.observation_space.shape

(4, 84, 84)

In [30]:
s1s.shape

torch.Size([4, 32, 1, 84, 84])

In [7]:
b = envs.observation_space.shape

In [5]:
b = torch.zeros((4,12,84,84))

In [9]:
a = (1, b[1:])
a

(1, (84, 84))